In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/cm/local/apps/python39/bin/python3 -m pip install --upgrade pip' command.


In [24]:
import torch
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

GPU: NVIDIA A100-SXM4-80GB is available.


In [15]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.5 MB 900 kB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/cm/local/apps/python39/bin/python3 -m pip install --upgrade pip' command.


In [7]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV  #finding best model
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

/jupyter/socse.user24/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import pandas as pd

# Raw URL of the CSV file
url = "https://raw.githubusercontent.com/ThejasHaridas/NLP_PROJECT_DEPRESSION/main/Augdata.csv"

# Read the CSV file
df = pd.read_csv(url)
df.head()


,PID,data,Label
0,dev_pid_1,"I enjoyed today, and I still am! Tomorrows dep...",moderate
1,dev_pid_2,I sorta tried to kill myself : I had a total b...,moderate
2,dev_pid_3,Best suicide method? : I like it quick and eas...,moderate
3,dev_pid_4,a story : I remember the time I'd get on my 3D...,moderate
4,dev_pid_5,The world only cares about beautiful people : ...,moderate


In [9]:
df["Label"].value_counts()

Label
moderate          2306
not depression    1830
severe            1440
Name: count, dtype: int64

In [10]:
# prompt: label encoder

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df["Label"] = le.fit_transform(df["Label"])
df.head()


,PID,data,Label
0,dev_pid_1,"I enjoyed today, and I still am! Tomorrows dep...",0
1,dev_pid_2,I sorta tried to kill myself : I had a total b...,0
2,dev_pid_3,Best suicide method? : I like it quick and eas...,0
3,dev_pid_4,a story : I remember the time I'd get on my 3D...,0
4,dev_pid_5,The world only cares about beautiful people : ...,0


In [17]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /jupyter/socse.user24/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /jupyter/socse.user24/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:

import nltk
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['.', ',', '!', '?', ';', ':'])

def remove_stopwords(text):
  """
  Removes stopwords from a given text.

  Args:
    text: A string containing the text.

  Returns:
    A string with stopwords removed.
  """
  tokens = nltk.word_tokenize(text)
  filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
  return ' '.join(filtered_tokens)

df['data'] = df['data'].apply(remove_stopwords)
df.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     /jupyter/socse.user24/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,PID,data,Label
0,dev_pid_1,enjoyed today still Tomorrows depression wait ...,0
1,dev_pid_2,sorta tried kill total breakdown fucking car p...,0
2,dev_pid_3,Best suicide method like quick easy deformitie...,0
3,dev_pid_4,story remember time 'd get 3DS play Nintendogs...,0
4,dev_pid_5,world cares beautiful people 'm born ugly 've ...,0


In [23]:
# For BERT:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

## Want DistilBERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [21]:
tokenized = df["data"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (826 > 512). Running this sequence through the model will result in indexing errors


In [25]:
print(tokenized)

0       [101, 5632, 2651, 2145, 4826, 2015, 6245, 3524...
1       [101, 4066, 2050, 2699, 3102, 2561, 12554, 823...
2       [101, 2190, 5920, 4118, 2066, 4248, 3733, 1336...
3       [101, 2466, 3342, 2051, 1005, 1040, 2131, 7605...
4       [101, 2088, 14977, 3376, 2111, 1005, 1049, 214...
                              ...                        
5571    [101, 1031, 1005, 1031, 1036, 1036, 1521, 5458...
5572    [101, 1031, 1005, 1031, 1036, 1036, 2342, 2393...
5573    [101, 1031, 1005, 1031, 1036, 1036, 24209, 130...
5574    [101, 1031, 1036, 1036, 1031, 1005, 6933, 1521...
5575    [101, 1031, 1005, 1031, 1036, 1036, 6224, 6040...
Name: data, Length: 5576, dtype: object


In [26]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [27]:
np.array(padded).shape

(5576, 2876)

In [28]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5576, 2876)

In [33]:
max_length = 512

# Truncate input_ids and attention_mask
input_ids = torch.tensor(input_ids) if not isinstance(input_ids, torch.Tensor) else input_ids
attention_mask = torch.tensor(attention_mask) if not isinstance(attention_mask, torch.Tensor) else attention_mask


# Proceed with the model inference
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)


In [34]:
print(attention_mask.size())

torch.Size([5576, 512])


In [35]:
features = last_hidden_states[0][:,0,:].numpy()

In [36]:
labels = df["Label"]

In [37]:
labels

0       0
1       0
2       0
3       0
4       0
       ..
5571    2
5572    2
5573    2
5574    2
5575    2
Name: Label, Length: 5576, dtype: int64

In [38]:
x_train, x_test, y_train, y_test = train_test_split(features, labels)

In [39]:

from sklearn import tree

dt_clf = tree.DecisionTreeClassifier()
dt_clf.fit(x_train, y_train)


DecisionTreeClassifier()

In [40]:
import numpy as np
pred= dt_clf.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.56      0.57      0.56       564
           1       0.49      0.49      0.49       458
           2       0.75      0.74      0.74       372

    accuracy                           0.59      1394
   macro avg       0.60      0.60      0.60      1394
weighted avg       0.59      0.59      0.59      1394

